In [1]:
import os
import zarr
import xarray as xr
from xgrads import open_CtlDataset

In [ ]:
gl_rmse = CtlDescriptor(file='gl/RMSEBAMH_20230216002023030300F.ctl')

In [ ]:
gl_rmse

In [ ]:
%%time

dset_gl_rmse = open_CtlDataset('gl/RMSEBAMH_20230216002023030300F.ctl')

In [ ]:
dset_lst[0]

In [ ]:
dset_gl_rmse

In [ ]:
dset_gl_rmse.vtmp925.isel(time=0).plot()

In [ ]:
%%time

df_gl_rmse = dset_gl_rmse.to_dataframe()

In [ ]:
df_gl_rmse

In [ ]:
%%time

df_gl_rmse.to_csv('gl/RMSEBAMH_20230216002023030300F.csv')

In [3]:
Regs = ['gl', 'hn', 'tr', 'hs', 'as']
#Exps = ['DTC', 'BAMH', 'BAMH0', 'X666']
Exps = ['X666']
Stats = ['VIES', 'RMSE', 'MEAN']
Tests = ['T1', 'T2', 'T3']

In [ ]:
def write_csv(df, filename):
    df.to_csv(filename)

In [ ]:
%%time

# Escrita paralela (ainda precisa juntar os arquivos depois...) - 12min

N = 8

for reg in Regs:
    for exp in Exps:
        for stat in Stats:
            print(reg, exp, stat)
            filein = str(stat) + str(exp) + '_20230216002023030300F.ctl'
            fnamein = os.path.join(reg, filein)
            dset = open_CtlDataset(fnamein).to_dataframe()
            parts = np.array_split(dset, N)
            fileout = str(stat) + str(exp) + '_20230216002023030300F.csv'
            fnameout = os.path.join(reg, fileout)
            #dset.to_csv(fnameout)
            Parallel(n_jobs=N)(delayed(write_csv)(part, f'./{reg}/{stat}{exp}_20230216002023030300F-{i}.csv') for i, part in enumerate(parts))

In [ ]:
%%time

# Escrita convencional, formato csv (1 arquivos por vez) - 23min

for reg in Regs:
    for exp in Exps:
        for stat in Stats:
            print(reg, exp, stat)
            filein = str(stat) + str(exp) + '_20230216002023030300F.ctl'
            fnamein = os.path.join(reg, filein)
            dset = open_CtlDataset(fnamein).to_dataframe()
            fileout = str(stat) + str(exp) + '_20230216002023030300F.csv'
            fnameout = os.path.join(reg, fileout)
            dset.to_csv(fnameout)

In [5]:
%%time

# Escrita convencional, formato zarr (1 arquivos por vez) - 9min

for reg in Regs:
    for exp in Exps:
        for stat in Stats:
            for test in Tests:
                print(reg, exp, stat, test)
                filein = str(stat) + str(exp) + '_20230216002023030300F.ctl'
                fnamein = os.path.join(test, reg, filein)
                fileout = str(stat) + str(exp) + '_20230216002023030300F.zarr'
                fnameout = os.path.join(test, reg, fileout)
                dset = open_CtlDataset(fnamein).to_zarr(fnameout, mode='w', consolidated=True)
                #print(fnameout)

gl X666 VIES T1
gl X666 VIES T2
gl X666 VIES T3
gl X666 RMSE T1
gl X666 RMSE T2
gl X666 RMSE T3
gl X666 MEAN T1
gl X666 MEAN T2
gl X666 MEAN T3
hn X666 VIES T1
hn X666 VIES T2
hn X666 VIES T3
hn X666 RMSE T1
hn X666 RMSE T2
hn X666 RMSE T3
hn X666 MEAN T1
hn X666 MEAN T2
hn X666 MEAN T3
tr X666 VIES T1
tr X666 VIES T2
tr X666 VIES T3
tr X666 RMSE T1
tr X666 RMSE T2
tr X666 RMSE T3
tr X666 MEAN T1
tr X666 MEAN T2
tr X666 MEAN T3
hs X666 VIES T1
hs X666 VIES T2
hs X666 VIES T3
hs X666 RMSE T1
hs X666 RMSE T2
hs X666 RMSE T3
hs X666 MEAN T1
hs X666 MEAN T2
hs X666 MEAN T3
as X666 VIES T1
as X666 VIES T2
as X666 VIES T3
as X666 RMSE T1
as X666 RMSE T2
as X666 RMSE T3
as X666 MEAN T1
as X666 MEAN T2
as X666 MEAN T3
CPU times: user 17.4 s, sys: 11.6 s, total: 29 s
Wall time: 8min 57s


In [ ]:
# Teste de abertura do arquivo zarr escrito

z = xr.open_zarr('./T2/gl/RMSEBAMH_20230216002023030300F.zarr')

In [ ]:
z

In [ ]:
z.umes925.isel(time=-1).plot()

In [ ]:
import intake
from intake.catalog import Catalog

In [ ]:
catalog = Catalog()

In [ ]:
def add_zarr_source(catalog, fnamein, fnameout):
    catalog.add(
        "dataset_name",
        driver="intake-xarray",
        args={
            "urlpath": fnameout,
            "metadata": {"original_data": fnamein}
        },
        description="Descrição do conjunto de dados Zarr"
    )

In [ ]:
input_directory = "/extra2/SCANTEC_XC50/SCANTEC-2.1.0/dataout/periodo/T1/gl"
output_directory = "/extra2/SCANTEC_XC50/SCANTEC-2.1.0/dataout/periodo/T1/gl/zarr"
list_of_input_files = [os.path.join(input_directory, filename) for filename in os.listdir(input_directory)]
list_of_output_files = [os.path.join(output_directory, "output_" + filename) for filename in os.listdir(input_directory)]

In [ ]:
list_of_output_files

In [ ]:
for fnamein, fnameout in zip(list_of_input_files, list_of_output_files):
    add_zarr_source(catalog, fnamein, fnameout)
    
catalog.save("teste.yml")   